In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torchvision import models
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim

class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000, aux_logits=True, init_weights=False):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)

        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:    # eval model lose this layer
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7

        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 1000 (num_classes)
        if self.training and self.aux_logits:   # eval model lose this layer
            return x, aux2, aux1
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

#inception结构
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)   # 保证输出大小等于输入大小
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)   # 保证输出大小等于输入大小
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

#辅助分类器
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.averagePool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)  # output[batch, 128, 4, 4]

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = self.averagePool(x)
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = torch.flatten(x, 1)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        x = F.dropout(x, 0.5, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x num_classes
        return x


class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x


In [3]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

#建立自己的dataset train_data
class MyDataset(Dataset):
    def __init__(self, image_path: list, image_class: list, transform=None):
        self.image_path = image_path
        self.image_class = image_class
        self.transform = transform

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, item):
        img = Image.open(self.image_path[item]).convert('RGB')
#         if img.mode != 'RGB':
#             raise ValueError("image: {} isn't RGB mode".format(self.image_path[item]))
        label = self.image_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [4]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

epoch_num=20
batch_size=32
lr=0.0003
path_photo='../input/bitmoji-faces-gender-recognition/BitmojiDataset/trainimages'
path_label='../input/bitmoji-faces-gender-recognition/train.csv'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# temp_name = os.listdir(path_photo)
df=pd.read_csv(path_label)
image_path=[]
label_path = []
for dir in os.listdir(path_photo):
#     print(dir)
    image_path.append(path_photo+'/'+str(dir))
    # 得到所有图片label
    if df[df.image_id == dir].iloc[0, 1]==-1:
        label_path.append(0)
    else:
        label_path.append(df[df.image_id == dir].iloc[0, 1])

image_path=np.array(image_path)
label_path=np.array(label_path)

train_path,val_path,train_label,val_label=train_test_split(image_path,label_path,test_size=0.2,random_state=1)
# print(len(train_path),len(val_path))
train_data=MyDataset(train_path,train_label,transform)
val_data=MyDataset(val_path,val_label,transform)
train_loader = DataLoader(train_data, batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size,shuffle=False)
print(train_loader)
print(val_loader)

In [5]:
net=GoogLeNet(num_classes=2, aux_logits=True, init_weights=True)
if torch.cuda.is_available():
    net=net.cuda()
# 参数量
n_p=sum(x.numel() for x in net.parameters())
print(n_p)


10309430


In [6]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), lr=lr) 

In [7]:
# 训练
import time
# iter=0
loss=0
acc=0
n=0
print('start training!')
start=time.time()
for epoch in range(epoch_num):
    for img,label in train_loader:
        # 将数据移到GPU上
        if torch.cuda.is_available():
            img, label = img.cuda(), label.cuda()
        # 先将optimizer梯度先置为0
        optimizer.zero_grad()
        out,aux_out1,aux_out2=net(img)
        # 计算loss,图的终点处
        loss0=criterion(out,label)
        loss1=criterion(aux_out1,label)
        loss2=criterion(aux_out2,label)
        loss=loss0+loss1*0.3+loss2*0.3

#         反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        prect =out.argmax(1)
        num_correct=(prect==label).sum()
        acc+=num_correct.item()
        n+=label.shape[0]
    print('epoch: {}, loss: {:.4} Acc: {:.6f}'.format(epoch, loss.data.item(),acc / n))

start= time.time()-start
print(start)
print("train over!")
if not os.path.exists('./model_save'):
    os.mkdir('./model_save')
torch.save(net.state_dict,os.path.join('./model_save/','epoch_{}_.pth'.format(epoch+1)))
#     if (epoch%50 == 0) & (epoch != 0):
#         i = epoch/50
#         torch.save(model, 'AlexNet%03d.pth'% i)

start training!


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


epoch: 0, loss: 0.2672 Acc: 0.665000
epoch: 1, loss: 0.07861 Acc: 0.797292
epoch: 2, loss: 0.1401 Acc: 0.851250
epoch: 3, loss: 0.3293 Acc: 0.876354
epoch: 4, loss: 0.0559 Acc: 0.897583
epoch: 5, loss: 0.008678 Acc: 0.913472
epoch: 6, loss: 0.001731 Acc: 0.924762
epoch: 7, loss: 0.05233 Acc: 0.933854
epoch: 8, loss: 0.0007975 Acc: 0.941157
epoch: 9, loss: 0.0001342 Acc: 0.947042
epoch: 10, loss: 5.144e-05 Acc: 0.951856
epoch: 11, loss: 0.0003765 Acc: 0.955868
epoch: 12, loss: 0.0005764 Acc: 0.959263
epoch: 13, loss: 0.0001176 Acc: 0.962173
epoch: 14, loss: 5.516e-05 Acc: 0.964694
epoch: 15, loss: 5.745e-05 Acc: 0.966901
epoch: 16, loss: 7.218e-05 Acc: 0.968848
epoch: 17, loss: 2.569e-05 Acc: 0.970579
epoch: 18, loss: 4.991e-05 Acc: 0.972127
epoch: 19, loss: 1.965e-05 Acc: 0.973521
6162.029001235962
train over!


In [8]:
import torch
# 模型评估
net.eval()
eval_loss=0
eval_acc=0
for img,label in val_loader:
    if torch.cuda.is_available():
        img=img.cuda()
        label=label.cuda()
    out=net(img)
    loss=criterion(out,label)
    eval_loss+=loss.item()*label.size(0)
    prect =out.argmax(1)
    num_correct=(prect==label).sum()
    eval_acc+=num_correct.item()
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(val_data)), eval_acc / (len(val_data))))

    

Test Loss: 0.000395, Acc: 1.000000


In [9]:
# 测试
import torch
import pandas as pd
testimg_path='../input/bitmoji-faces-gender-recognition/BitmojiDataset/testimages'
submission_path='../input/bitmoji-faces-gender-recognition/sample_submission.csv'

cnt=0
test_df=pd.read_csv(submission_path)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# net.eval()
with torch.no_grad():
    parent_list=os.listdir(testimg_path)
    parent_list.sort()
#     print(parent_list)
    for img_name in parent_list:
        img_path = os.path.join(testimg_path, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        if torch.cuda.is_available():
            img=img.cuda()
        img = img.unsqueeze(0)
        prect = int(net(img).argmax(1))
        if prect==0:
            prect=-1
#         print(prect)
        test_df['is_male'].iloc[cnt:cnt+1]=str(prect)
        cnt+=1
# test_df.iloc[:,1:2]
print(test_df)    
test_df.to_csv('./sample_submission.csv',index=False,header=True)
print('test_over!')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


      image_id is_male
0     3000.jpg      -1
1     3001.jpg      -1
2     3002.jpg      -1
3     3003.jpg       1
4     3004.jpg      -1
...        ...     ...
1079  4079.jpg      -1
1080  4080.jpg       1
1081  4081.jpg       1
1082  4082.jpg      -1
1083  4083.jpg       1

[1084 rows x 2 columns]
test_over!
